**Water Usage Report Submittal Percentage Script**

Consists of 3 main blocks:

1. Connecting to the ReportManager, 1542 server, Report DB databse, which hosts all the ewrims flat files
1. Linking to the Main_sheet of the Watershed Demand Dataset Paths spreadsheet on SharePoint
1. Generic Function Definition for generating_reporting_summary--this function calculates the percentage of eligible rights
    that reported for a given watershed; its input arguments are:
    * pods_df--this is the POD list for the watershed after the completion of the GIS manual review 
    * watershed_code--this is the 2-character code for the watershed, "NV" for the Navarro Watershed for example
    * conn--this is the connection mechanism to the server and database with the flat files
    * output_folder--this is the folder where we will export all the summary tables generated by this script
1. Function call that iterates through each each watershed listed in the Watershed Demand Dataset Paths spreadsheet

In [65]:
# STEP 1: Define all paths and load the Watershed Demand Dataset Paths ("Driver") spreadsheet
import os
import pandas as pd

username = os.getenv("USERNAME")

# Set working directory to the DWRAT_DataScraping\Demand folder
project_root = fr"C:\Users\{username}\Documents\GitHub\DWRAT_DataScraping\Demand"
os.chdir(project_root)

# Define the path to the Watershed Demand Dataset Paths ("Driver") spreadsheet
sharepoint_path = fr"C:\Users\{username}\Water Boards\Supply and Demand Assessment - Documents\Program Watersheds\4. Demand Data Tracking\Watershed_Demand_Dataset_Paths.xlsx"

# Load the driver spreadsheet
driver_df = pd.read_excel(sharepoint_path, skiprows=1, sheet_name = "Main_Sheet")
driver_df.columns

# Ensure that the 
driver_df['POD_APPLICATION_NUMBER_SPREADSHEET_PATH'] = (
    driver_df['POD_APPLICATION_NUMBER_SPREADSHEET_PATH']
    .astype(str)
    .str.strip()
)


In [18]:
# STEP 2: Connect to ReportManager, 1542 server and the ReportDB database
import pyodbc

conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=reportmanager,1542;'
    'DATABASE=ReportDB;'
    'Trusted_Connection=yes;'
)

In [ ]:
# Pick a single watershed to debug
row = driver_df.iloc[11]  # Or use .loc if you want by watershed_code

watershed_code = row['ID']
sheet_name = row['POD_APPLICATION_NUMBER_WORKSHEET_NAME']
partial_path = row['POD_APPLICATION_NUMBER_SPREADSHEET_PATH']

username = os.getenv("USERNAME")
base_path = fr"C:\Users\{username}\Water Boards\Supply and Demand Assessment - Documents"
full_path = os.path.join(base_path, partial_path)

print(f"🧪 Watershed: {watershed_code}")
print(f"🧪 Path: {full_path}")

pods_df = pd.read_excel(full_path, sheet_name=sheet_name)
pods_df.head()

pods_df['APPLICATION_NUMBER'] = pods_df['APPLICATION_NUMBER'].astype(str).str.strip()
pods_df = pods_df.drop_duplicates(subset='APPLICATION_NUMBER') # remove duplicate water rights
pods_df = pods_df.rename(columns={'APPLICATION_NUMBER': 'app_number'}) # Rename the APPLICATION_NUMBER column
pods_df = pods_df[['app_number']] # Remove all columns except for app_number

query = """
SELECT DISTINCT Application_Number AS app_number, Effective_Date
FROM ReportDB.FLAT_FILE.ewrims_flat_file
"""
ewrims_df = pd.read_sql(query, conn)
ewrims_df['app_number'] = ewrims_df['app_number'].astype(str).str.strip()

joined_df = pd.merge(pods_df, ewrims_df, on='app_number', how='inner')
joined_df['Effective_Date'] = pd.to_datetime(joined_df['Effective_Date'], errors='coerce')
joined_df.head()

for year in range(2017, 2025):
    cutoff = pd.to_datetime(f'{year - 1}-12-31')
    joined_df[f'Eligible_{year}'] = joined_df['Effective_Date'].apply(
        lambda x: 'Y' if pd.notnull(x) and x <= cutoff else 'N'
    )

app_numbers = joined_df['app_number'].unique().tolist()
query_reports = f"""
SELECT APPL_ID AS app_number, YEAR
FROM ReportDB.FLAT_FILE.ewrims_water_use_report
WHERE YEAR BETWEEN 2017 AND 2024
AND APPL_ID IN ({','.join("'" + x + "'" for x in app_numbers)})
"""
report_df = pd.read_sql(query_reports, conn)
report_df['app_number'] = report_df['app_number'].astype(str).str.strip()
report_df['YEAR'] = report_df['YEAR'].astype(int) # Convert the Year column to an integer type

year = 2024
eligible_set = set(joined_df[joined_df[f'Eligible_{year}'] == 'Y']['app_number'])
reported_set = set(report_df[report_df['YEAR'] == year]['app_number'])

print(f"✅ {year} — Eligible: {len(eligible_set)}, Reported: {len(reported_set)}, Matched: {len(eligible_set & reported_set)}")



In [53]:
# STEP 3: Define the generate_reporting_summary function
def generate_reporting_summary(pods_df, watershed_code, conn, output_folder):
    # 1. Clean and standardize APPLICATION_NUMBER
    pods_df['APPLICATION_NUMBER'] = pods_df['APPLICATION_NUMBER'].astype(str).str.strip()
    pods_df = pods_df.drop_duplicates(subset='APPLICATION_NUMBER') # Remove duplicate water rights
    pods_df = pods_df.rename(columns={'APPLICATION_NUMBER': 'app_number'}) 
    pods_df = pods_df[['app_number']] # Remove all columns except for app_number
    
    # 2. Load Effective_Date from ewrims_flat_file
    query_flat_file = """
    SELECT DISTINCT Application_Number AS app_number, Effective_Date
    FROM ReportDB.FLAT_FILE.ewrims_flat_file
    """
    ewrims_df = pd.read_sql(query_flat_file, conn)
    ewrims_df['app_number'] = ewrims_df['app_number'].astype(str).str.strip()

    # 3. Inner join to get Effective_Date
    joined_df = pd.merge(pods_df, ewrims_df, on='app_number', how='inner')
    joined_df['Effective_Date'] = pd.to_datetime(joined_df['Effective_Date'], errors='coerce')
    joined_df['app_number'] = joined_df['app_number'].astype(str).str.strip()
   

    # 4. Create eligibility flags
    for year in range(2017, 2025):
        cutoff = pd.to_datetime(f'{year-1}-12-31')
        joined_df[f'Eligible_{year}'] = joined_df['Effective_Date'].apply(
            lambda x: 'Y' if pd.notnull(x) and x <= cutoff else 'N'
        )

    # 5. Create temp table of app_numbers
    app_numbers = joined_df['app_number'].dropna().unique().tolist()
    cursor = conn.cursor()
    cursor.execute("IF OBJECT_ID('tempdb..#AppNumbers') IS NOT NULL DROP TABLE #AppNumbers")
    cursor.execute("CREATE TABLE #AppNumbers (app_number NVARCHAR(50))")
    for app in app_numbers:
        cursor.execute("INSERT INTO #AppNumbers (app_number) VALUES (?)", app)
    conn.commit()

    # 6. Query water_use_report for matching applications
    query_reports = """
    SELECT APPL_ID AS app_number, YEAR
    FROM ReportDB.FLAT_FILE.ewrims_water_use_report
    WHERE YEAR BETWEEN 2017 AND 2024
    AND APPL_ID IN (SELECT app_number FROM #AppNumbers)
    """
    report_df = pd.read_sql(query_reports, conn)
    report_df['app_number'] = report_df['app_number'].astype(str).str.strip()
    report_df['YEAR'] = report_df['YEAR'].astype(int) # Convert the Year column to an integer type

    # 7. Calculate reporting summary
    summary = []
    for year in range(2017, 2025):
        eligible_col = f'Eligible_{year}'
        eligible_set = set(joined_df[joined_df[eligible_col] == 'Y']['app_number'])
        reported_set = set(report_df[report_df['YEAR'] == year]['app_number'])
        matched = eligible_set & reported_set
        summary.append({
            'Year': year,
            'Eligible_Count': len(eligible_set),
            'Reported_Count': len(matched),
            'Reporting_Percentage': round(len(matched) / len(eligible_set) * 100, 2) if eligible_set else 0.0
        })

    # 8. Export summary
    summary_df = pd.DataFrame(summary)
    output_path = os.path.join(output_folder, f'reporting_summary_df_{watershed_code}.csv')
    summary_df.to_csv(output_path, index=False)
    print(f"✓ Exported {watershed_code} summary to {output_path}")
    
    print(f"Sample app_number from joined_df: {joined_df['app_number'].iloc[0]} ({type(joined_df['app_number'].iloc[0])})")
    print(f"Sample app_number from report_df: {report_df['app_number'].iloc[0]} ({type(report_df['app_number'].iloc[0])})")

    # Return the individual summary for collection
    return summary_df



In [68]:
# STEP 4: Iterate over driver_df and call the function
all_summaries = []
results = []

for _, row in driver_df.iterrows():

    watershed_code = row['ID']
    sheet_name = row['POD_APPLICATION_NUMBER_WORKSHEET_NAME']

    
    output_folder = os.path.join(os.getcwd(), "OutputData")
    os.makedirs(output_folder, exist_ok=True)

    username = os.getenv("USERNAME")
    partial_path = row['POD_APPLICATION_NUMBER_SPREADSHEET_PATH']

      # ⚠️ Validate partial_path before attempting to use it
    if pd.isna(partial_path) or not isinstance(partial_path, str):
        print(f"⚠️ Skipping {watershed_code} — invalid or missing POD path: {repr(partial_path)}")
        results.append({'Watershed': watershed_code, 'Status': 'Skipped (Invalid path)'})
        continue

    base_path = fr"C:\Users\{username}\Water Boards\Supply and Demand Assessment - Documents"
    full_path = os.path.join(base_path, partial_path)

    print(f"🚀 Processing {watershed_code}...")

    try:
        pods_df = pd.read_excel(io=full_path, sheet_name=sheet_name)
        summary_df = generate_reporting_summary(pods_df, watershed_code, conn, output_folder)

        # Add watershed code as a column in each summary
        summary_df['Watershed'] = watershed_code
        all_summaries.append(summary_df)

        results.append({'Watershed': watershed_code, 'Status': 'Success'})
    except Exception as e:
        print(f"❌ Failed for {watershed_code}: {e}")
        results.append({'Watershed': watershed_code, 'Status': 'Failed', 'Error': str(e)})

# Combine all per-watershed summaries into a master file
master_df = pd.concat(all_summaries, ignore_index=True)
master_df.to_csv("OutputData/master_reporting_summary.csv", index=False)


🚀 Processing RR...
❌ Failed for RR: Worksheet named 'RR_pod_points_Merge_filtered_PA' not found
🚀 Processing NV...


C:\Users\palemi\AppData\Local\Temp\1\ipykernel_20160\187834552.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ewrims_df = pd.read_sql(query_flat_file, conn)
C:\Users\palemi\AppData\Local\Temp\1\ipykernel_20160\187834552.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  report_df = pd.read_sql(query_reports, conn)


✓ Exported NV summary to C:\Users\PAlemi\Documents\GitHub\DWRAT_DataScraping\Demand\OutputData\reporting_summary_df_NV.csv
Sample app_number from joined_df: A016624 (<class 'str'>)
Sample app_number from report_df: A009618 (<class 'str'>)
🚀 Processing NR...


C:\Users\palemi\AppData\Local\Temp\1\ipykernel_20160\187834552.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ewrims_df = pd.read_sql(query_flat_file, conn)
C:\Users\palemi\AppData\Local\Temp\1\ipykernel_20160\187834552.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  report_df = pd.read_sql(query_reports, conn)


✓ Exported NR summary to C:\Users\PAlemi\Documents\GitHub\DWRAT_DataScraping\Demand\OutputData\reporting_summary_df_NR.csv
Sample app_number from joined_df: A000631 (<class 'str'>)
Sample app_number from report_df: A000631 (<class 'str'>)
🚀 Processing BC...


...

✓ Exported TD summary to C:\Users\PAlemi\Documents\GitHub\DWRAT_DataScraping\Demand\OutputData\reporting_summary_df_TD.csv
Sample app_number from joined_df: A004881 (<class 'str'>)
Sample app_number from report_df: A004881 (<class 'str'>)
🚀 Processing TR...


C:\Users\palemi\AppData\Local\Temp\1\ipykernel_20160\187834552.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ewrims_df = pd.read_sql(query_flat_file, conn)
C:\Users\palemi\AppData\Local\Temp\1\ipykernel_20160\187834552.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  report_df = pd.read_sql(query_reports, conn)


✓ Exported TR summary to C:\Users\PAlemi\Documents\GitHub\DWRAT_DataScraping\Demand\OutputData\reporting_summary_df_TR.csv
Sample app_number from joined_df: A001029 (<class 'str'>)
Sample app_number from report_df: A001205 (<class 'str'>)
🚀 Processing SFT...
❌ Failed for SFT: [Errno 2] No such file or directory: 'C:\\Users\\PAlemi\\Water Boards\\Supply and Demand Assessment - Documents\\nan'
🚀 Processing MT...
❌ Failed for MT: [Errno 2] No such file or directory: 'C:\\Users\\PAlemi\\Water Boards\\Supply and Demand Assessment - Documents\\nan'
🚀 Processing PC ...


C:\Users\palemi\AppData\Local\Temp\1\ipykernel_20160\187834552.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ewrims_df = pd.read_sql(query_flat_file, conn)
C:\Users\palemi\AppData\Local\Temp\1\ipykernel_20160\187834552.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  report_df = pd.read_sql(query_reports, conn)


✓ Exported PC  summary to C:\Users\PAlemi\Documents\GitHub\DWRAT_DataScraping\Demand\OutputData\reporting_summary_df_PC .csv
Sample app_number from joined_df: A000533 (<class 'str'>)
Sample app_number from report_df: A000533 (<class 'str'>)


In [ ]:
# Import the Russian River POD List and whittle it down
import pandas as pd
rr_pod_file_path = r'C:\Users\palemi\Water Boards\Supply and Demand Assessment - Documents\Program Watersheds\1. Watershed Folders\Test Watersheds\Test_Russian\Data\GIS_Preprocessing\TR_GIS_Preprocessing_2025-03-17.xlsx'
rr_pods = pd.read_excel(rr_pod_file_path, sheet_name = 'Final_POD_List')

# Remove all columns except for APPLICATION_NUMBER

rr_pods = rr_pods[['APPLICATION_NUMBER']]
# Rename the APPLICATION_NUMBER field in rr_pods to app_num
rr_pods.rename(columns={'APPLICATION_NUMBER': 'app_number'}, inplace=True)

# Grab unique application numbers
rr_pods = rr_pods.drop_duplicates()

In [ ]:
import pandas as pd

# Inner Join rr_pods to the ewrims_flat_file SQL server table on the APPLICATION_NUMBER column
query1 = """
SELECT DISTINCT
    Application_Number AS app_number,
    Effective_Date
FROM ReportDB.FLAT_FILE.ewrims_flat_file
"""

query2 = """
SELECT DISTINCT *, Application_Number AS app_number
FROM ReportDB.FLAT_FILE.ewrims_flat_file
"""

# Read the SQL query results into DataFrames
ewrims_flat_file = pd.read_sql(query1, conn)
ewrims_flat_file_all = pd.read_sql(query2, conn)

# Preview the result
print(ewrims_flat_file.head())
print(ewrims_flat_file_all.head())

# Perform inner join on APPLICATION_NUMBER
rr_pods_flat_file= pd.merge(rr_pods, ewrims_flat_file, on='app_number', how='inner')
rr_pods_flat_file_all = pd.merge(rr_pods,ewrims_flat_file_all, on = 'app_number', how = 'inner')

print(f"Number of matched water rights: {len(rr_pods_flat_file)}")
rr_pods_flat_file.head(20)

In [ ]:
# Ensure Effective_Date is a datetime object
rr_pods_flat_file['Effective_Date'] = pd.to_datetime(rr_pods_flat_file['Effective_Date'], errors='coerce')

# Define years for eligibility tagging
years = list(range(2017, 2025))

# Apply eligibility logic based on Effective_Date
for year in years:
    cutoff = pd.to_datetime(f'{year - 1}-12-31')
    rr_pods_flat_file[f'Eligible_{year}'] = rr_pods_flat_file['Effective_Date'].apply(
        lambda x: 'Y' if pd.notnull(x) and x <= cutoff else 'N'
    )

# Preview relevant columns
columns_to_show = ['app_number', 'Effective_Date'] + [f'Eligible_{y}' for y in years]
rr_pods_flat_file[columns_to_show].head(10)

# Count Eligible Rights By Year--create a summary table

# Initialize an empty list to collect summary data
# Initialize an empty list to collect summary data
eligibility_counts = []

# Loop through each Eligible column and count the Ys
for year in range(2017, 2025):
    col_name = f'Eligible_{year}'
    count = rr_pods_flat_file[col_name].value_counts().get('Y', 0)
    eligibility_counts.append({'Year': year, 'Eligible_Count': count})  # <-- needs to be indented!


eligibility_summary_df = pd.DataFrame(eligibility_counts)
eligibility_summary_df

In [ ]:
test_query = "SELECT TOP 5 * FROM ReportDB.FLAT_FILE.ewrims_water_use_report"
test_df = pd.read_sql(test_query, conn)

# Print all column names
print(test_df.columns.tolist())

In [ ]:
# Import ewrims_flat_file_water_report (but just the rights in the Russian River)

# Get unique app_numbers in the Russian River as a list of strings
app_numbers = rr_pods_flat_file['app_number'].dropna().unique().tolist()

# Create a DataFrame of unique app_numbers
app_numbers_df = pd.DataFrame({'app_number': app_numbers})


cursor = conn.cursor()

# Drop temp table if it exists (to be safe)
cursor.execute("IF OBJECT_ID('tempdb..#AppNumbers') IS NOT NULL DROP TABLE #AppNumbers")
cursor.execute("CREATE TABLE #AppNumbers (app_number NVARCHAR(50))")

# Insert values into the temp table
for app in app_numbers:
    cursor.execute("INSERT INTO #AppNumbers (app_number) VALUES (?)", app)

conn.commit()

query = """
SELECT Distinct
 R.APPL_ID As Application_Number, 
 R.Year
FROM ReportDB.FLAT_FILE.ewrims_water_use_report R
INNER JOIN #AppNumbers A ON R.APPL_ID = A.app_number
WHERE YEAR>= 2017
"""

report_df = pd.read_sql(query, conn)
print(f"Retrieved {len(report_df)} rows from filtered report table.")


In [ ]:
# CALCULATING REPORTING PERCENTAGES

# Initialize an empty list to contain the results
reporting_summary = []

# year = 2024 # Use this for a single iteration of the loop
for year in range(2017,2025):
    eligible_col = f'Eligible_{year}'

    # Step 1: Get all eligible rights for this year
    eligible_df = rr_pods_flat_file[rr_pods_flat_file[eligible_col] == 'Y']
    eligible_apps = set(eligible_df['app_number'])

    # Step 2: Get all reported rights for this year from the report table
    report_df['Year'] = report_df['Year'].astype(int) # Convert the Year column to an integer type
    reported_df_year = report_df[report_df['Year'] == year]
    reported_apps = set(reported_df_year['Application_Number'])

    #Step 3: Intersect the 2 datasets to find the eligible rights that submitted reports
    matched_reports = eligible_apps & reported_apps

    # Step 4: Count the reports and calculate the percentages
    eligible_count = len(eligible_apps)
    reported_count = len(matched_reports)
    percentage = round(reported_count/eligible_count * 100, 2) if eligible_count > 0 else 0

    # Step 5: Store the results in the reporting_summary list
    reporting_summary.append({
        'Year': year,
        'Eligible_Count': eligible_count,
        'Reported_Count': reported_count,
        'Reporting_Percentage': percentage
    })

# Convert the reporting_summary list into a DataFrame
reporting_summary_df = pd.DataFrame(reporting_summary)

# Ensure the output directory exists
output_dir = "OutputData"
os.makedirs(output_dir, exist_ok=True)

# Export report_summary_df as a csv
reporting_summary_df.to_csv(f"{output_dir}/reporting_summary_df_RR.csv", index=False)

In [ ]:
import os
os.getcwd()
